# 2.2) 트위터 API로 연관 키워드 분석하기

- [<Step1. API 호출> : 트위터 API로 데이터 가져오기](#<Step1.-API-호출>-:-트위터-API로-데이터-가져오기)
    - [API 데이터로 데이터 프레임 생성하기]
- [<Step2. 추출> : 키워드 추출](#<Step2.-추출>-:-키워드-추출)
    - [텍스트 데이터 전처리]
    - [nltk, konlpy를 이용한 키워드 추출]
- [<Step3. 분석> : 연관 분석을 이용한 키워드 분석](#<Step3.-분석>-:-연관-분석을-이용한-키워드-분석)
    - [연관 키워드 추출하기]
    - [단어 빈도 추출하기]
- [<Step4. 시각화> : 연관 키워드 네트워크 시각화](#<Step4.-시각화>-:-연관-키워드-네트워크-시각화)
    - [연관 키워드 네트워크 시각화]

In [1]:
# -*- coding: utf-8 -*-

# 주피터 노트북을 실행한 브라우저에서 바로 그림을 볼 수 있게끔 만드는 것
#브라우저 내부(inline)에 바로 그려지도록 해주는 코드
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# <Step1. API 호출> : 트위터 API로 데이터 가져오기

### [API 데이터로 데이터 프레임 생성하기]
- anaconda prompt 혹은 Terminal에서 아래와 같은 패키지를 설치
- (env_name) `pip install tweepy`

In [2]:
# tweepy 3 버전으로 설치
#!pip install tweepy==3.10.0

In [3]:
import tweepy
tweepy.__version__

'3.10.0'

In [4]:
# 발급 완료된 키 입력
CONSUMER_KEY ="1guUu16Re83rAexS7VhsCAQv0"
CONSUMER_SECRET ="DMlEHpONvAo5ZPtpDt1wZPFrHplrQIYijOGgD2suu2p9HW0zQz"
ACCESS_TOKEN_KEY = "169786640-wXppNaveQqbXzOkf6ls3Nt4QQPs04JQUZkoypIjv"
ACCESS_TOKEN_SECRET ="9ubAWb5RKlIyREa3jc6NM6JUnLp8RILbieFrmEMRDONt0" 

# 개인정보 인증을 요청하는 Handler
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)

# 인증 요청을 수행
auth.set_access_token(ACCESS_TOKEN_KEY, ACCESS_TOKEN_SECRET)

# twitter API를 사용하기 위한 준비
api = tweepy.API(auth)

In [5]:
auth

In [6]:
api

#### '손흥민' 키워드 검색

In [7]:
# twitter API를 사용하여 '손흥민' 이 포함된 트윗들을 크롤링한 뒤, 'user_mentions'와 'hashtags' 속성값 추출
keyword = "축구"
# 특정 키워드 크롤링은  search() 함수 수행, 결과가 리스트 형태로 출력
tweets = api.search(keyword)
# entities() 함수로 특정 속성의 메타데이터에 접근하여 정보 추출
for tweet in tweets:
    print(tweet.text)
    print(tweet.entities['user_mentions']) # 사용자 댓글
    print(tweet.entities['hashtags']) # 해시태그 
    print(tweet.created_at) # 트윗이 생성된 UTC 시간

RT @yonhaptweet: 한국 여자축구, 사상 첫 아시안컵 결승 진출…필리핀에 2-0 완승 https://t.co/COr0uBtKLS
[{'screen_name': 'yonhaptweet', 'name': '연합뉴스', 'id': 147451838, 'id_str': '147451838', 'indices': [3, 15]}]
[]
2022-02-04 06:49:57
RT @Jaemyung_Lee: &lt;한국 여자 축구가 31년 만에 아시안컵 결승에 진출했습니다. 축하드립니다. 월드컵 10회 연속 진출에 이은 쾌거입니다. 비인기종목의 설움에도 선수들이 보인 성과에 감복합니다. 결승전 선전을 기원합니다.&gt;
https:…
[{'screen_name': 'Jaemyung_Lee', 'name': '이재명', 'id': 106379129, 'id_str': '106379129', 'indices': [3, 16]}]
[]
2022-02-04 06:48:38
RT @yonhaptweet: 한국 여자축구, 사상 첫 아시안컵 결승 진출…필리핀에 2-0 완승 https://t.co/COr0uBtKLS
[{'screen_name': 'yonhaptweet', 'name': '연합뉴스', 'id': 147451838, 'id_str': '147451838', 'indices': [3, 15]}]
[]
2022-02-04 06:48:38
RT @SBS8news: 한국 여자축구, 6일 중국과 아시안컵 결승…첫 우승 도전 #SBS뉴스 https://t.co/1MlGG3xpAr
[{'screen_name': 'SBS8news', 'name': 'SBS 뉴스', 'id': 91946403, 'id_str': '91946403', 'indices': [3, 12]}]
[{'text': 'SBS뉴스', 'indices': [46, 52]}]
2022-02-04 06:47:41
RT @KJH_officialtwt: [#HWANAIR]
잠시 후 9시, JTBC #전설체전 에 재환이

#### 데이터 프레임 형태로 수집

In [8]:
# 크롤링된 데이터를 저장할 데이터 프레임 생성
columns = ['created', 'tweet_text']
df = pd.DataFrame(columns=columns)

# 크롤링을 수행할 갯수를 입력
max_tweets = 100

# Cursor 객체를 사용하여 크롤링 수행
searched_tweets = [status for status in tweepy.Cursor(api.search, q=keyword).items(max_tweets)]

# ‘손흥민’이 포함된 1000개의 트윗들에서, ‘text’, ‘created_at’ 정보를 데이터 프레임으로 저장
for tweet in searched_tweets:
    tweet_json = tweet._json
    tweet_text = tweet_json['text']
    created = tweet_json['created_at']
    row = [created, tweet_text]
    series = pd.Series(row, index=df.columns) #  created, tweet_text로 시리즈를 생성
    df = df.append(series, ignore_index=True) # 생성된 시리즈를 데이터프레임에 추가
    
# 데이터 프레임 상위 5개 출력
df.head()

,created,tweet_text
0,Fri Feb 04 06:49:57 +0000 2022,"RT @yonhaptweet: 한국 여자축구, 사상 첫 아시안컵 결승 진출…필리핀에..."
1,Fri Feb 04 06:48:38 +0000 2022,RT @Jaemyung_Lee: &lt;한국 여자 축구가 31년 만에 아시안컵 결승...
2,Fri Feb 04 06:48:38 +0000 2022,"RT @yonhaptweet: 한국 여자축구, 사상 첫 아시안컵 결승 진출…필리핀에..."
3,Fri Feb 04 06:47:41 +0000 2022,"RT @SBS8news: 한국 여자축구, 6일 중국과 아시안컵 결승…첫 우승 도전 ..."
4,Fri Feb 04 06:47:39 +0000 2022,"RT @KJH_officialtwt: [#HWANAIR]\n잠시 후 9시, JTBC..."


In [9]:
# 데이터 프레임을 csv 로 저장
# df.to_csv("tweet_temp.csv", index=False)

# <Step2. 추출> : 키워드 추출

### [텍스트 데이터 전처리]
- 한글 문자열을 기준으로 키워드 추출 수행
- tweet_text 데이터에서 한글 문자열만을 추출하여 ko_text 생성

In [10]:
import re

# 텍스트 정제 함수 : 한글 및 띄어쓰기 이외의 문자는 전부 제거
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글의 정규표현식
    result = hangul.sub('', text)
    return result

# 텍스트 정제 함수text_cleaning()를 ‘tweet_text’ 피처에 이를 적용, 한글과 띄어쓰기만 추출하여 ko_text 피처 생성
df['ko_text'] = df['tweet_text'].apply(lambda x: text_cleaning(x))
df.head()

,created,tweet_text,ko_text
0,Fri Feb 04 06:49:57 +0000 2022,"RT @yonhaptweet: 한국 여자축구, 사상 첫 아시안컵 결승 진출…필리핀에...",한국 여자축구 사상 첫 아시안컵 결승 진출필리핀에 완승
1,Fri Feb 04 06:48:38 +0000 2022,RT @Jaemyung_Lee: &lt;한국 여자 축구가 31년 만에 아시안컵 결승...,한국 여자 축구가 년 만에 아시안컵 결승에 진출했습니다 축하드립니다 월드컵 회 ...
2,Fri Feb 04 06:48:38 +0000 2022,"RT @yonhaptweet: 한국 여자축구, 사상 첫 아시안컵 결승 진출…필리핀에...",한국 여자축구 사상 첫 아시안컵 결승 진출필리핀에 완승
3,Fri Feb 04 06:47:41 +0000 2022,"RT @SBS8news: 한국 여자축구, 6일 중국과 아시안컵 결승…첫 우승 도전 ...",한국 여자축구 일 중국과 아시안컵 결승첫 우승 도전 뉴스
4,Fri Feb 04 06:47:39 +0000 2022,"RT @KJH_officialtwt: [#HWANAIR]\n잠시 후 9시, JTBC...",잠시 후 시 전설체전 에 재환이가 출연합니다 축구돌 김째니 발재간의 향연으로...


### [konlpy를 이용한 명사 키워드 추출]
- 명사만 추출한 후 불용어 및 한글자 키워드 제거

In [11]:
from konlpy.tag import Okt
from collections import Counter

# 불용어 사전(korean_stopwords.txt)의 텍스트 문서 읽어 stopwords 리스트 생성
korean_stopwords_path ="korean_stopwords.txt"
with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords] # strip()으로 공백 및 줄바꿈 제거

# 품사 중 명사만 추출하여 한글자 키워드 및 불용어 제거
# Okt(Open Korean Text)는 트위터에서 만든 오픈소스 한국어 처리기인 twitter-korean-text를 이어받아 만들고 있는 프로젝트
def get_nouns(x):
    nouns_tagger = Okt() # Okt()클래스 선언
    nouns = nouns_tagger.nouns(x)
    
    # 한글자 키워드 제거
    nouns = [noun for noun in nouns if len(noun) > 1]
    
    # 불용어를 제거
    nouns = [noun for noun in nouns if noun not in stopwords]
    
    return nouns

# ‘ko_text’ 피처에 get_nouns() 함수를 적용하여 불용어 및 한글자를 제거한 명사 추출하여 'nouns' 피처 생성
df['nouns'] = df['ko_text'].apply(lambda x: get_nouns(x))
print(df.shape) # 행과 열의 갯수를 튜플로 반환
df.head()


(100, 4)


,created,tweet_text,ko_text,nouns
0,Fri Feb 04 06:49:57 +0000 2022,"RT @yonhaptweet: 한국 여자축구, 사상 첫 아시안컵 결승 진출…필리핀에...",한국 여자축구 사상 첫 아시안컵 결승 진출필리핀에 완승,"[한국, 여자, 축구, 사상, 아시안컵, 결승, 진출, 필리핀, 완승]"
1,Fri Feb 04 06:48:38 +0000 2022,RT @Jaemyung_Lee: &lt;한국 여자 축구가 31년 만에 아시안컵 결승...,한국 여자 축구가 년 만에 아시안컵 결승에 진출했습니다 축하드립니다 월드컵 회 ...,"[한국, 여자, 축구, 아시안컵, 결승, 진출, 축하, 월드컵, 연속, 진출, 쾌거..."
2,Fri Feb 04 06:48:38 +0000 2022,"RT @yonhaptweet: 한국 여자축구, 사상 첫 아시안컵 결승 진출…필리핀에...",한국 여자축구 사상 첫 아시안컵 결승 진출필리핀에 완승,"[한국, 여자, 축구, 사상, 아시안컵, 결승, 진출, 필리핀, 완승]"
3,Fri Feb 04 06:47:41 +0000 2022,"RT @SBS8news: 한국 여자축구, 6일 중국과 아시안컵 결승…첫 우승 도전 ...",한국 여자축구 일 중국과 아시안컵 결승첫 우승 도전 뉴스,"[한국, 여자, 축구, 중국, 아시안컵, 결승, 우승, 도전, 뉴스]"
4,Fri Feb 04 06:47:39 +0000 2022,"RT @KJH_officialtwt: [#HWANAIR]\n잠시 후 9시, JTBC...",잠시 후 시 전설체전 에 재환이가 출연합니다 축구돌 김째니 발재간의 향연으로...,"[전설, 체전, 재환, 출연, 축구, 재간, 향연, 당황, 재화니, 매력, 퐁당]"


# <Step3. 분석> : 연관 분석을 이용한 키워드 분석

### [연관 키워드 추출하기]
### [연관 규칙]
- 연관규칙 (아프리오리, Apriori 알고리즘) https://blog.naver.com/zzz90zzz/221807210555
- 연관규칙은 비지도학습으로 대규모 거래 데이터로 부터 함께 구매될 규칙을 도출하여 고객이 특정 상품 구매시 이와 연관성이 높은 상품을 추천하는 것
- Apriori 알고리즘 : 간단한 성능 측정치를 이용해 거대한 DB에서 데이터간의 연관성을 찾는 알고리즘
#### [ 연관규칙에서 사용하는 3가지 통계척도 ]
- 1. 지지도 (support)  : 특정 아이템이 데이터에서 발생하는 빈도 
- 2. 신뢰도 (confidence) : 두 아이템의 연관규칙이 유용한 규칙일 가능성의 척도 
- 3. 향상도 (lift) : 두 아이템의 연관 규칙이 우연인지 아닌지를 나타내는 척도 

### Apriori 라이브러리 설치

In [13]:
##### 연관 분석 연습
from apyori import apriori

transactions = [
    ['손흥민', '시소코'],
    ['손흥민', '케인'],
    ['손흥민', '케인', '포체티노']
]

# 연관 분석을 수행
results = list(apriori(transactions))
for result in results:
    print(result)

RelationRecord(items=frozenset({'손흥민'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'손흥민'}), confidence=1.0, lift=1.0)])
RelationRecord(items=frozenset({'시소코'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'시소코'}), confidence=0.3333333333333333, lift=1.0)])
RelationRecord(items=frozenset({'케인'}), support=0.6666666666666666, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'케인'}), confidence=0.6666666666666666, lift=1.0)])
RelationRecord(items=frozenset({'포체티노'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'포체티노'}), confidence=0.3333333333333333, lift=1.0)])
RelationRecord(items=frozenset({'손흥민', '시소코'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'손흥민', '시소코'}), confidence=0.3333333333333333, lift=1.0), Or

In [14]:
# 지지도 0.5, 신뢰도 0.6, 향상도 1.0 이상이면서 (손흥민, 케인) 처럼 규칙의 크기가 2 이하인 규칙을 추출
list(apriori(transactions,
             min_support=0.5,
             min_confidence=0.6,
             min_lift=1.0,
             max_length=2))

[RelationRecord(items=frozenset({'손흥민'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'손흥민'}), confidence=1.0, lift=1.0)]),
 RelationRecord(items=frozenset({'케인'}), support=0.6666666666666666, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'케인'}), confidence=0.6666666666666666, lift=1.0)]),
 RelationRecord(items=frozenset({'케인', '손흥민'}), support=0.6666666666666666, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'케인', '손흥민'}), confidence=0.6666666666666666, lift=1.0), OrderedStatistic(items_base=frozenset({'손흥민'}), items_add=frozenset({'케인'}), confidence=0.6666666666666666, lift=1.0), OrderedStatistic(items_base=frozenset({'케인'}), items_add=frozenset({'손흥민'}), confidence=1.0, lift=1.0)])]

#### 트위터 연관 키워드 분석

In [15]:
# 트랜잭션 데이터를 추출
transactions = df['nouns'].tolist()
print(transactions)

[['한국', '여자', '축구', '사상', '아시안컵', '결승', '진출', '필리핀', '완승'], ['한국', '여자', '축구', '아시안컵', '결승', '진출', '축하', '월드컵', '연속', '진출', '쾌거', '종목', '설움', '선수', '성과', '감복', '결승전', '선전', '기원'], ['한국', '여자', '축구', '사상', '아시안컵', '결승', '진출', '필리핀', '완승'], ['한국', '여자', '축구', '중국', '아시안컵', '결승', '우승', '도전', '뉴스'], ['전설', '체전', '재환', '출연', '축구', '재간', '향연', '당황', '재화니', '매력', '퐁당'], ['한국', '여자', '축구', '사상', '아시안컵', '결승', '진출', '필리핀', '완승'], ['속보', '여자', '축구', '사상', '아시안컵', '결승', '진출', '위키트리'], ['한국', '여자', '축구', '사상', '아시안컵', '결승', '진출', '필리핀', '완승'], ['한국', '여자', '축구', '중국', '아시안컵', '결승', '우승', '도전', '뉴스'], ['패티', '만남', '어플', '스타킹', '축구', '축구선수', '울산', '아줌마', '동영상', '애무', '김제', '맛집', '조건', '만남', '출장', '만남', '출장'], ['한국', '여자', '축구', '사상', '아시안컵', '결승', '진출', '필리핀', '완승'], ['축구', '정우'], ['댄스', '팬티', '프레', '걸그룹', '노출', '공연', '파티쉐', '여성', '축구', '나잇', '오피', '검스', '계절', '노출증', '만남'], ['고창', '출장', '경산', '훈녀', '모바일', '수원', '유흥업소', '골든', '일드', '반사', '인스타', '친구', '일상', '부천', '영암군', '유러', '피안', '레터', '번역', '축구', '

In [16]:
# 지지도 0.05, 신뢰도 0.1, 향상도 5 이상인 것을 추출
#list(apriori(transactions))
list(apriori(transactions,
                       min_support=0.05,
                       min_confidence=0.1,
                       min_lift=5,
                       max_length=2))

[RelationRecord(items=frozenset({'결승전', '감복'}), support=0.11, ordered_statistics=[OrderedStatistic(items_base=frozenset({'감복'}), items_add=frozenset({'결승전'}), confidence=1.0, lift=9.090909090909092), OrderedStatistic(items_base=frozenset({'결승전'}), items_add=frozenset({'감복'}), confidence=1.0, lift=9.090909090909092)]),
 RelationRecord(items=frozenset({'기원', '감복'}), support=0.11, ordered_statistics=[OrderedStatistic(items_base=frozenset({'감복'}), items_add=frozenset({'기원'}), confidence=1.0, lift=9.090909090909092), OrderedStatistic(items_base=frozenset({'기원'}), items_add=frozenset({'감복'}), confidence=1.0, lift=9.090909090909092)]),
 RelationRecord(items=frozenset({'선수', '감복'}), support=0.11, ordered_statistics=[OrderedStatistic(items_base=frozenset({'감복'}), items_add=frozenset({'선수'}), confidence=1.0, lift=7.142857142857142), OrderedStatistic(items_base=frozenset({'선수'}), items_add=frozenset({'감복'}), confidence=0.7857142857142857, lift=7.142857142857142)]),
 RelationRecord(items=frozenset